In [1]:
%load_ext autoreload
%autoreload 2

from copy import deepcopy
import torch
import os
import sys
sys.path.append("/home/msst/repo/Quantization")
from qlib.vector_quantization.kmeans.kmeans_job import kmeas_job, KmeasJobParams


torch.backends.cuda.matmul.allow_tf32 = True
DEVICE = 'cuda'
DTYPE = torch.float32

model_name='Llama2-7b-hf'
# model_name='Llama-3.2-1B'

In [2]:
path_to_per_block_fp = f'/mnt/ssd_storage/ml/weights/vc_data/{model_name}/per_block_fp'

path_to_hess = f'/mnt/ssd_storage/ml/weights/vc_data/{model_name}/xtx/{model_name}_xtx.pth'
xtx = torch.load(path_to_hess, weights_only=True)

#path_to_model_grads = '/home/msst/repo/Quantization/weights/Llama-3.2-1B-grads-slimpajama-hf'
#grads_model = AutoModelForCausalLM.from_pretrained(path_to_model_grads)

path_to_save = f'/mnt/ssd_storage/ml/weights/vc_data/{model_name}/kmeans/'

In [3]:
args = KmeasJobParams(
    layer_name = None,
    matrix = None,
    path_to_save=path_to_save,
    vector_dim=8,
    codebook_size=2**15, #256,
    num_iters=10, #10,
    scale_type='OUTL2', #' #None,#'OUTSTD',#'OUTL2',
    weighting_type="PERCOORD",
    distance_type='MSE',
    init_type='RANDOM',
    batch_size=2**14,
    matrix_block_size=None,
	use_absolute_coordinates=True,
	haar_decomposition_level=None,
	use_incoherence_processing=False,
    eps=1e-8,
)

In [4]:
block_names = sorted(
    [name for name in os.listdir(path_to_per_block_fp) if name.startswith('model.layers')],
    key=lambda name: int(name.split('.')[-1])
)


for block_name in block_names[:3]:
    block = torch.load(
        os.path.join(path_to_per_block_fp, block_name)
    )
    matrices_and_names = []
    for module_name, module in block.named_modules():
        if module.__class__ == torch.nn.Linear:
            full_module_name = '.'.join([block_name, module_name])
            matrices_and_names.append([module.weight.data, full_module_name])

    for matrix, name in matrices_and_names:
        hess = xtx[name].expand(matrix.shape).clone()
        #hess = torch.ones_like(matrix)
        
        params = deepcopy(args)
        params.layer_name = name
        params.matrix = matrix.to(DTYPE)
        params.hess = hess.to(DTYPE)

        res = kmeas_job(params, device=DEVICE)
        print(res)

/tmp/ipykernel_32570/97967701.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  block = torch.load(
100%|██████████| 10/10 [01:49<00:00, 10.93s/iter]


{'layer_name': 'model.layers.0.self_attn.q_proj', 'mse': 0.0003602021897677332}


100%|██████████| 10/10 [01:37<00:00,  9.71s/iter]


{'layer_name': 'model.layers.0.self_attn.k_proj', 'mse': 0.0004511828301474452}


100%|██████████| 10/10 [01:36<00:00,  9.67s/iter]


...

{'layer_name': 'model.layers.2.mlp.gate_proj', 'mse': 0.0006900613661855459}


100%|██████████| 10/10 [03:43<00:00, 22.33s/iter]


{'layer_name': 'model.layers.2.mlp.up_proj', 'mse': 0.0006002670852467418}


100%|██████████| 10/10 [03:43<00:00, 22.33s/iter]


{'layer_name': 'model.layers.2.mlp.down_proj', 'mse': 0.0006076947320252657}
